In [1]:
from abc import ABC, abstractmethod

In [2]:
class Runnable(ABC):

    @abstractmethod
    def invoke(input_data):
        pass

In [5]:
import random

# first component
class NakliLLM(Runnable): # inheriting runnable

    def __init__(self):
        print("LLM created")

    def invoke(self, prompt):
        response_list = [
            "Delhi is the capital of India.",
            "IPL is a cricket league.",
            "AI stands for artificial intelligence"
        ]

        return {"response": random.choice(response_list)}
    
    def predict(self, prompt):
        response_list = [
            "Delhi is the capital of India.",
            "IPL is a cricket league.",
            "AI stands for artificial intelligence"
        ]

        return {"response": random.choice(response_list)}

In [6]:
llm = NakliLLM()

LLM created


In [11]:
# second component
class NakliPromptTemplate(Runnable):

    def __init__(self, template, input_variables):
        self.template = template
        self.input_variables = input_variables

    def invoke(self, input_dict):
        return self.template.format(**input_dict)

    def format(self, input_dict): # will be deprecated use invoke instead
        return self.template.format(**input_dict)

In [24]:
class NakliStrOutputParser(Runnable):

    def __init__(self):
        pass

    def invoke(self, input_data):
        return input_data["response"]

In [25]:
# now we have standardized both the components, its time to make chains

In [26]:
class RunnableConnector(Runnable):

    def __init__(self, runnable_list):
        self.runnable_list = runnable_list

    def invoke(self, input_data):

        for runnable in self.runnable_list: # can chain multiple components
            input_data = runnable.invoke(input_data) 
            # making output of current runnable component as 
            # input to next runnable component
        
        return input_data

In [27]:
template = NakliPromptTemplate(
    template="Write a {length} poem about {topic}",
    input_variables=["length", "topic"]
)

In [28]:
llm = NakliLLM()

LLM created


In [29]:
parser = NakliStrOutputParser()

In [30]:
chain = RunnableConnector([template, llm, parser])

In [31]:
chain.invoke({"length": "long", "topic": "India"})

'IPL is a cricket league.'

In [32]:
template1 = NakliPromptTemplate(
    template="Write a joke about {topic}",
    input_variables=["topic"]
)

In [33]:
template2 = NakliPromptTemplate(
    template="Explain the following joke {response}",
    input_variables=["response"]
)

In [34]:
llm = NakliLLM()

LLM created


In [35]:
parser = NakliStrOutputParser()

In [36]:
# print the joke
chain1 = RunnableConnector([template1, llm])

In [37]:
# chain1.invoke({"topic": "AI"})

{'response': 'AI stands for artificial intelligence'}

In [38]:
chain2 = RunnableConnector([template2, llm, parser])

In [39]:
# chain2.invoke({"response": "This is a joke."})

'IPL is a cricket league.'

In [40]:
final_chain = RunnableConnector([chain1, chain2]) # combining both the chains

In [41]:
final_chain.invoke({"topic": "cricket"})

'AI stands for artificial intelligence'